In [1]:
from pathlib import Path
import random
from pprint import pprint
submission_path = Path("effnetv2b2_unet_eff1_ensemble")
submission_path.mkdir(exist_ok=True)
submission_assets_path = submission_path / "assets"
submission_assets_path.mkdir(exist_ok=True)
submission_assets_path_effnet = submission_assets_path / "unet-eff1"
submission_assets_path_effnet.mkdir(exist_ok=True)

submission_assets_path_effnetv2 = submission_assets_path / "effnetv2b2"
submission_assets_path_effnetv2.mkdir(exist_ok=True)

In [2]:
import shutil
import glob
for f in glob.glob(r'../models/timm-efficientnet-b1-*_wo_ca.pth'):
    print(f)
    f1 = Path(f).name
    shutil.copyfile(f,submission_assets_path_effnet/f1)
    
    
for f in glob.glob(r'../models/tf_efficientnetv2_b2-*.pth'):
    print(f)
    f1 = Path(f).name
    shutil.copyfile(f,submission_assets_path_effnetv2/f1)
    

../models\timm-efficientnet-b1-fold0_wo_ca.pth
../models\tf_efficientnetv2_b2-fold0.pth


In [3]:
%%file $submission_path/cloud_model.py

from pathlib import Path
import random
from pprint import pprint

import torch
import torch.nn as nn
from timm.models.efficientnet import *
import segmentation_models_pytorch as smp
import rasterio
import segmentation_models_pytorch as smp
import numpy as np
import pandas as pd
import time
import os
import torchvision

# These transformations will be passed to our model class
class CloudDataset(torch.utils.data.Dataset):

    def __init__(self, chip_ids_df, 
                 x_path = '../data/train_features/', 
                 y_path= '../data/train_labels/', 
                 bands=[4,3,2],transforms=None):
        self.data = chip_ids_df
        self.data_path = x_path
        self.label_path = y_path
        self.bands = bands
        self.transforms = transforms
        self.max_values = {4:23104,3:26096,2:27600,8:19568}

    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, idx):
        img = self.data.iloc[idx]
        chip_id = img.chip_id
        imgs = []
        for b in self.bands:
            pth = f'{self.data_path}/{chip_id}/B0{b}.tif'
            with rasterio.open(pth) as img_file:
                img = img_file.read(1).astype(float)
                img = (img/2**16).astype(np.float32)
                imgs.append(img)
        x_arr= np.stack(imgs,axis=-1)
        
        x_arr = np.transpose(x_arr, [2, 0, 1])
        sample = {"chip_id": chip_id, "chip": x_arr}

        return sample
    


 


import torch
import torch.nn as nn
import torch.nn.functional as F
import timm


from segmentation_models_pytorch.unet.decoder import UnetDecoder
from segmentation_models_pytorch.base import SegmentationHead

class UnetEffNetV2(nn.Module):
    def __init__(self,params):
        super(UnetEffNetV2, self).__init__()
        norm_cfg = dict(type='BN', requires_grad=True)
        self.backbone = timm.create_model('tf_efficientnetv2_b2', features_only=True, 
                                          out_indices=[0,1,2,3],pretrained=True)
        self.decode_head = UnetDecoder(
                            encoder_channels=[16, 32, 56, 120],
                            decoder_channels=[16, 32, 56, 120],
                            n_blocks=4,
                            use_batchnorm=True,
                            center=False,
                            attention_type=None
                        )
        
        self.segment_classifier = SegmentationHead(56,1,upsampling=4)

        
    def forward(self,image):
        image = image[:,0:3]
        x = self.backbone(image)
        x=self.decode_head(*x)
        x=self.segment_classifier(x)
        x = F.interpolate(x, image.shape[-2:], mode="bilinear", align_corners=True)
        return x
    

class UNetEFF1_4CH(nn.Module):
    def __init__(self,params):
        super(UNetEFF1_4CH, self).__init__()

        aux_params=dict(
                        pooling='avg',             # one of 'avg', 'max'
                        dropout=0.3,               # dropout ratio, default is None
                        activation=None,      # activation function, default is None
                        classes=1,
                    ) 
        self.unet = smp.Unet(
                    encoder_name=params['backbone'],        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
                    encoder_weights=params['weights'],     # use `imagenet` pre-trained weights for encoder initialization
                    in_channels=4,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                    decoder_attention_type= None,                      # model output channels (number of classes in your dataset)
                    classes=1,aux_params=aux_params
                    )



    # @torch.cuda.amp.autocast()
    def forward(self, image):
        batch_size = len(image)
        mask,logit = self.unet(image)
        return mask

Writing effnetv2b2_unet_eff1_ensemble/cloud_model.py


In [4]:
%%file $submission_path/main.py
import os
from pathlib import Path
from typing import List
from loguru import logger
import numpy as np
import pandas as pd
from tifffile import imwrite
from tqdm.auto import tqdm
import torch
import typer
import gc

from cloud_model import UNetEFF1_4CH,CloudDataset,UnetEffNetV2
import torchvision

ROOT_DIRECTORY = Path("/codeexecution")
PREDICTIONS_DIRECTORY = ROOT_DIRECTORY / "predictions"
ASSETS_DIRECTORY = ROOT_DIRECTORY / "assets"
DATA_DIRECTORY = ROOT_DIRECTORY / "data"
INPUT_IMAGES_DIRECTORY = DATA_DIRECTORY / "test_features"


# Make sure the smp loader can find our torch assets because we don't have internet!
os.environ["TORCH_HOME"] = str(ASSETS_DIRECTORY / "torch")
GPU=torch.cuda.is_available()

hparams = {
    "backbone": 'timm-efficientnet-b1',
    "weights": "noisy-student",
}

def get_metadata(features_dir: os.PathLike, bands: List[str]):
    """
    Given a folder of feature data, return a dataframe where the index is the chip id
    and there is a column for the path to each band's TIF image.

    Args:
        features_dir (os.PathLike): path to the directory of feature data, which should have
            a folder for each chip
        bands (list[str]): list of bands provided for each chip
    """
    chip_metadata = pd.DataFrame(index=[f"{band}_path" for band in bands])
    chip_ids = (
        pth.name for pth in features_dir.iterdir() if not pth.name.startswith(".")
    )

    for chip_id in chip_ids:
        chip_bands = [features_dir / chip_id / f"{band}.tif" for band in bands]
        chip_metadata[chip_id] = chip_bands

    return chip_metadata.transpose().reset_index().rename(columns={"index": "chip_id"})

def getEffv2Model(hparams):
    unet_model = UnetEffNetV2(hparams)
    unet_model.cuda()
    return unet_model

def getEffb1Model4ch(hparams):
    unet_model = UNetEFF1_4CH(hparams)
    unet_model.cuda()
    return unet_model

def prediction_step(data, models,th=0.5,predictions_dir='./'):
   
    chip_ids = data['chip_id']
    images = data['chip'].float()
    if GPU:
        images = images.cuda()

    preds = np.zeros((images.shape[0],images.shape[2],images.shape[3]))
    
    images = torch.stack([images, torchvision.transforms.functional.hflip(images),
                            torchvision.transforms.functional.vflip(images),
                            ], 0)
    n, bs, c, h, w = images.size()
    images = images.view(-1, c, h, w)
    
    for model in models:
        model.eval()
        with torch.no_grad():
            mask = model(images)
            ##TTA
            
            probs1, probs2, probs3 = torch.split(mask, bs)

            
            probs2 = torchvision.transforms.functional.hflip(probs2)
            probs3 = torchvision.transforms.functional.vflip(probs3)
            
            
            mask =  (1/3)*probs1 + (1/3)*probs2 + (1/3)*probs3
            mask = mask.sigmoid()
            preds += mask[:,0].cpu().numpy()
    preds /= len(models)
    preds = ((preds>0.5)*1).astype(np.uint8)
    
    for ix in range(len(chip_ids)):
        chip_id = chip_ids[ix]
        output_path = predictions_dir / f"{chip_id}.tif"
        imwrite(output_path, preds[ix], dtype=np.uint8)
        


params = {
    "backbone": 'timm-efficientnet-b1',
    "weight": "noisy-student",
}

def main(
    assets_dir_path: Path = ASSETS_DIRECTORY,
    test_features_dir: Path = DATA_DIRECTORY / "test_features",
    predictions_dir: Path = PREDICTIONS_DIRECTORY,
    bands: List[str] = ["B02", "B03", "B04", "B08"],):

    if not test_features_dir.exists():
        raise ValueError(
            f"The directory for test feature images must exist and {test_features_dir} does not exist"
        )
    predictions_dir.mkdir(exist_ok=True, parents=True)
    
    logger.info("Loading model")
    # Explicitly set where we expect smp to load the saved resnet from just to be sure
    torch.hub.set_dir(assets_dir_path / "torch/hub")
    
    model_paths = assets_dir_path.glob('*.pth')
    models = []
    
    assets_dir_path1 = assets_dir_path/'effnetv2b2'
    model_paths = assets_dir_path1.glob('*.pth')
    
    for mp in model_paths:
        print('Loading Model from Path',mp)
        f = torch.load(mp, map_location=torch.device('cpu'))
        model = getEffv2Model(params)
        model.load_state_dict(f)
        model.cuda()
        models.append(model)
        del f
        gc.collect()
        
    assets_dir_path2 = assets_dir_path/'unet-eff1'
    model_paths = assets_dir_path2.glob('*.pth')
    
    for mp in model_paths:
        print('Loading Model from Path',mp)
        f = torch.load(mp, map_location=torch.device('cpu'))
        model = getEffb1Model4ch(hparams)
        model.load_state_dict(f)
        model.cuda()
        models.append(model)
        del f
        gc.collect()
        
    torch.cuda.empty_cache()
    
    logger.info("Finding chip IDs")
    chip_id_metadata = get_metadata(test_features_dir, bands)
    
    logger.info(f"Found {len(chip_id_metadata)} test chip_ids. Generating predictions.")
    
    #create dataset and data loader
    testDataSet = val_dataset = CloudDataset(chip_id_metadata, 
                                             x_path= test_features_dir, 
                                             bands = [4,3,2,8])
    testDataLoader = torch.utils.data.DataLoader(
                        testDataSet,
                        batch_size=16,num_workers=4,shuffle=False,pin_memory=False
                        )
    
    for i,data in tqdm(enumerate(testDataLoader),total=len(testDataLoader)):
        prediction_step(data,models,predictions_dir=predictions_dir)

    logger.success(f"Inference complete.")


if __name__ == "__main__":
    typer.run(main)


Writing effnetv2b2_unet_eff1_ensemble/main.py
